In [18]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
(x_train_all, y_train_all), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()


In [20]:
x_train_all.shape

(60000, 28, 28)

In [21]:
x_test.shape

(10000, 28, 28)

In [22]:
np.unique(y_train_all)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [23]:
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

In [24]:
x_train_all.dtype

dtype('uint8')

In [25]:
# 标准化处理
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(55000, -1) / 255.0).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(x_valid.reshape(5000, -1) / 255.0).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(x_test.reshape(10000, -1) / 255.0).reshape(-1, 28, 28, 1)

In [26]:
x_train_scaled.shape

(55000, 28, 28, 1)

In [27]:
x_train_scaled.dtype

dtype('float64')

In [28]:
def make_dataset(data, target, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((data, target))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

In [29]:
batch_size = 64
epochs = 30
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)

In [30]:
# 创建model
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters = 32,
                             kernel_size = 3,
                             padding = 'same',
                             activation = 'relu',
                             input_shape = (28, 28, 1)))
        

model.add(keras.layers.Conv2D(filters = 32,
                             kernel_size = 3,
                             padding = 'same',
                             activation = 'relu'))
          
# 池化          
model.add(keras.layers.MaxPool2D())          

model.add(keras.layers.Conv2D(filters = 64,
                             kernel_size = 3,
                             padding = 'same',
                             activation = 'relu'))
model.add(keras.layers.Conv2D(filters = 64,
                             kernel_size = 3,
                             padding = 'same',
                             activation = 'relu'))

model.add(keras.layers.MaxPool2D())   

model.add(keras.layers.Conv2D(filters = 128,
                             kernel_size = 3,
                             padding = 'same',
                             activation = 'relu'))

model.add(keras.layers.Conv2D(filters = 128,
                             kernel_size = 3,
                             padding = 'same',
                             activation = 'relu'))

model.add(keras.layers.MaxPool2D())

# flatten之后放到全连接
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                  

In [32]:
eval_dataset = make_dataset(x_valid_scaled, y_valid, epochs=1, batch_size=32, shuffle=False)

In [33]:
history = model.fit(train_dataset,
                   steps_per_epoch=x_train_scaled.shape[0] // batch_size,
                   epochs=30,
                   validation_data=eval_dataset)

Epoch 1/30
859/859 [==============================] - 20s 9ms/step - loss: 0.4610 - acc: 0.8300 - val_loss: 0.3347 - val_acc: 0.8674
Epoch 2/30
859/859 [==============================] - 7s 8ms/step - loss: 0.2771 - acc: 0.8977 - val_loss: 0.2597 - val_acc: 0.9016
Epoch 3/30
859/859 [==============================] - 7s 9ms/step - loss: 0.2258 - acc: 0.9163 - val_loss: 0.2346 - val_acc: 0.9108
Epoch 4/30
859/859 [==============================] - 8s 9ms/step - loss: 0.1957 - acc: 0.9273 - val_loss: 0.2232 - val_acc: 0.9194
Epoch 5/30
859/859 [==============================] - 7s 9ms/step - loss: 0.1674 - acc: 0.9375 - val_loss: 0.2316 - val_acc: 0.9208
Epoch 6/30
859/859 [==============================] - 8s 9ms/step - loss: 0.1471 - acc: 0.9448 - val_loss: 0.2373 - val_acc: 0.9200
Epoch 7/30
859/859 [==============================] - 7s 8ms/step - loss: 0.1296 - acc: 0.9510 - val_loss: 0.2484 - val_acc: 0.9220
Epoch 8/30
859/859 [==============================] - 8s 9ms/step - loss: 0

In [34]:
test_dataset = make_dataset(x_test_scaled, y_test, 1, 32)

In [35]:
model.evaluate(test_dataset)

313/313 [==============================] - 1s 4ms/step - loss: 0.5964 - acc: 0.9152


[0.5964409112930298, 0.9151999950408936]